<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- 다양한 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 검증합니다.
- 특정 모델을 GridSearchCV를 사용해 성능을 최적화합니다.

# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [1]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
# 데이터 불러오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/titanic.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [3]:
# 데이터 살펴보기
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# 기초통계정보 확인
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) Title 변수 추가**

In [5]:
# 호칭 추출
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [6]:
# 대표 호칭 이외는 Others로 변경
main_tit_list = ['Mr', 'Miss', 'Mrs']
data.loc[data['Title'].isin(main_tit_list)==False, ['Title']] = 'Others'

**2) 결측치 처리**

In [7]:
# Age 중앙값으로 채우기
data['Age'].fillna(data.groupby('Title')['Age'].transform('median'), inplace=True)

In [8]:
# Embarked 최빈값으로 채우기
data['Embarked'] = data['Embarked'].fillna('S')

**3) AgeGroup 변수 추가**

In [9]:
# 변수 추가
bins = [0, 16, 25, 35, 60, 100]
data['AgeGroup'] = pd.cut(data['Age'], 
                          bins=bins, 
                          labels=['0_16', '17_25', '26_35', '36_60', '61_100'])

**4) 불필요한 변수 제거**

In [10]:
# 불필요한 열 제거
drop_cols = ['PassengerId', 'Name', 'Age', 'Ticket', 'Cabin']
data.drop(drop_cols, axis=1, inplace=True)

**5) 가변수화**

In [11]:
# 가변수화
dummy_vars = ['Title', 'AgeGroup', 'Pclass', 'Sex', 'Embarked']

for v in dummy_vars:
    dummies = pd.get_dummies(data[v], prefix=v, drop_first=True)
    data = pd.concat([data, dummies], axis=1)
    data.drop(v, axis=1, inplace=True)

**6) x, y 분리**

In [12]:
# target 확인
target = 'Survived'

# 데이터 분리
x = data.drop(target, axis=1)
y = data[target]

**7) 정규화**

In [13]:
# 정규화
x = (x - x.min()) - (x.max() - x.min())

**8) 학습용, 평가용 데이터 분리**

In [14]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델 검증

- 여러 알고리즘으로 모델을 만들고 K-Fold CV로 성능을 검증합니다.
- 하이퍼파라미터를 설정하지 않고 진행합니다.
- 각 모델의 성능 정보를 수집해 마지막에 비교합니다.

## 4.1.KNN

- KNN 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [15]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [16]:
# 선언하기
model = KNeighborsClassifier()

In [17]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [18]:
# 결과수집
result = {}
result['KNN'] = cv_score.mean()

## 4.2.Decision Tree

- Decision Tree 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022을 지정합니다.

In [19]:
# 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [20]:
# 선언하기
model = DecisionTreeClassifier(random_state=2022)

In [21]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [22]:
# 결과수집
result['Decision Tree'] = cv_score.mean()

## 4.3.Logistic Regression

- Logistic Regression 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.

In [23]:
# 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [24]:
# 선언하기
model = LogisticRegression()

In [25]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train)

In [26]:
# 결과수집
result['Logistic Regression'] = cv_score.mean()

## 4.4.SVM

- SVM 알고리즘으로 모델링하고 K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022을 지정합니다.

In [27]:
# 불러오기
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [28]:
# 선언하기
model = SVC(random_state=2022)

In [29]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [30]:
# 결과수집
result['SVM'] = cv_score.mean()

## 4.5.Random Forest

- Random Forest 알고리즘으로 모델링하고  K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022을 지정합니다.

In [31]:
# 불러오기
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [32]:
# 선언하기
model = RandomForestClassifier(random_state=2022)

In [33]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [34]:
# 결과수집
result['Random Forest'] = cv_score.mean()

## 4.6.XGBoost

- XGBoost 알고리즘으로 모델링하고  K-Fold CV로 성능을 검증합니다.
- 모델 선언시 random_state=2022를 지정합니다.

In [35]:
# 불러오기
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [36]:
# 선언하기
model = XGBClassifier(random_state=2022, eval_metric='logloss')

In [37]:
# 성능검증
cv_score = cross_val_score(model, x_train, y_train, cv=10)

In [38]:
# 결과수집
result['XGBoost'] = cv_score.mean()

# 5.결과 확인

In [39]:
# 평가 결과 확인
print('='*40)
for m_name, m_score in result.items() :
    print(m_name, ':', m_score)
print('='*40)

KNN : 0.7723502304147465
Decision Tree : 0.7802867383512545
Logistic Regression : 0.7929419354838709
SVM : 0.6405017921146954
Random Forest : 0.8011520737327189
XGBoost : 0.8076292882744495


# 6.모델 최적화

- SVM 모델을 GridSearchCV를 사용해 최적화합니다.
- 파라미터 범위
    - C: [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    - gamma: [0.001, 0.1, 1, 10]}
- GridSearchCV 옵션
    - cv=5

In [40]:
# 1단계: 불러오기
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [41]:
# 2단계: 선언하기
model_svm = SVC(random_state=2022)

param = {'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
         'gamma': [0.001, 0.1, 1, 10]}

model = GridSearchCV(model_svm,
                     param,
                     cv=5,
                     scoring='accuracy')

In [42]:
# 3단계: 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=2022),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'gamma': [0.001, 0.1, 1, 10]},
             scoring='accuracy')

In [43]:
# 결과확인
print(model.best_params_)
print(model.best_score_)

{'C': 100, 'gamma': 0.001}
0.8138322580645161


In [44]:
# 4단계: 예측하기
y_pred = model.predict(x_test)

In [45]:
# 5단계: 평가하기
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       168
           1       0.76      0.76      0.76       100

    accuracy                           0.82       268
   macro avg       0.81      0.81      0.81       268
weighted avg       0.82      0.82      0.82       268

